In [ ]:
#!/usr/local/bin/python3.6

import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from scapy.all import *

packets = rdpcap('./pcap/test.pcapng')

In [ ]:
#traitement des paquets ###############################################


i = 0
tableau = []
for y in range(len(packets)+1):
    tableau.append(["" for y in range(27)])


for packet in packets:
    #packet.show()
    ###[ Ethernet ]###
    tableau[i][0] = packet[0].dst
    tableau[i][1] = packet[0].src
    tableau[i][2] = packet[0].type
    ###[ IP ]###
    tableau[i][3] = packet[1].version
    tableau[i][4] = packet[1].ihl
    tableau[i][5] = packet[1].tos
    tableau[i][6] = packet[1].len
    tableau[i][7] = packet[1].id
    tableau[i][8] = packet[1].flags
    tableau[i][9] = packet[1].frag
    tableau[i][10] = packet[1].ttl
    tableau[i][11] = packet[1].proto
    tableau[i][12] = packet[1].chksum
    tableau[i][13] = packet[1].src
    tableau[i][14] = packet[1].dst
    ###[ TCP ]###
    tableau[i][15] = packet[2].sport
    tableau[i][16] = packet[2].dport
    tableau[i][17] = packet[2].seq
    tableau[i][18] = packet[2].ack
    tableau[i][19] = packet[2].dataofs
    tableau[i][20] = packet[2].reserved
    tableau[i][21] = packet[2].flags
    tableau[i][22] = packet[2].window
    tableau[i][23] = packet[2].chksum
    tableau[i][24] = packet[2].urgptr
    if packet[2].options is not None:
        tableau[i][25] = packet[2].options
    else:
        tableau[i][25] = ""
    ###[ Raw ]###
    if "Raw" in packet:
        tableau[i][26] = packet[3].load
    else:
        tableau[i][26] = ""

    print(tableau[i][26])
    i += 1
    



In [ ]:
'''
print(packets[4854][2].load)
print("#######")
print(tableau)
'''

for test in tableau:
    
    if re.search('\.\.\/', str(test)):
        print(test)
    elif re.search('Nikto', str(test)):
        print(test)
    elif re.search('cmd=', str(test)):
        print(test)

In [ ]:
for i in range(20):
    #afficher 20 paquets et leur menace 

In [ ]:
# reshape éventuel ###############################################

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [ ]:
#mettre dans une variable la menace 0 ou 1

In [ ]:
# création du modèle 

model = Sequential()
model.add(Dense(512, input_shape=(784,)))
model.add(Activation('relu')) # An "activation" is just a non-linear function applied to the output
                              # of the layer above. Here, with a "rectified linear unit",
                              # we clamp all values below 0 to 0.
                           
model.add(Dropout(0.2))   # Dropout helps protect the model from memorizing or "overfitting" the training data
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax')) # This special "softmax" activation among other things,
                                 # ensures the output is a valid probaility distribution, that is
                                 # that its values are all non-negative and sum to 1.

In [ ]:
# compilation du model ###############################################

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# training  ###############################################

model.fit(X_train, Y_train, batch_size=128, nb_epoch=4, verbose=1,
         validation_data=(X_test, Y_test))

In [ ]:
  # evualtion ###############################################
         
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# résultat ###############################################

# The predict_classes function outputs the highest probability class
# according to the trained classifier for each input example.
predicted_classes = model.predict_classes(X_test)

# Check which items we got right / wrong
correct_indices = np.nonzero(predicted_classes == y_test)[0]
incorrect_indices = np.nonzero(predicted_classes != y_test)[0]